In [1]:
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dropout, Flatten, Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

In [2]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    shear_range=0.2,
    zoom_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    validation_split=0.2
)
train_data = train_datagen.flow_from_directory(
    'D:/Yogesh/collage/G12/train',
    target_size=(80, 80),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)
validation_data = train_datagen.flow_from_directory(
    'D:/Yogesh/collage/G12/train',
    target_size=(80, 80),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

Found 61942 images belonging to 2 classes.
Found 15484 images belonging to 2 classes.


In [3]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_data = test_datagen.flow_from_directory(
    'D:/Yogesh/collage/G12/test',
    target_size=(80, 80),
    batch_size=8,
    class_mode='categorical'
)

Found 7472 images belonging to 2 classes.


In [4]:
base_model = InceptionV3(
    include_top=False,
    weights='imagenet',
    input_tensor=Input(shape=(80, 80, 3))
)

In [5]:
head_model = base_model.output
head_model = Flatten()(head_model)
head_model = Dense(64, activation='relu')(head_model)
head_model = Dropout(0.5)(head_model)
head_model = Dense(2, activation='softmax')(head_model)

model = Model(inputs=base_model.input, outputs=head_model)

for layer in base_model.layers:
    layer.trainable = False


In [6]:
checkpoint = ModelCheckpoint(
    'D:/Yogesh/collage/G12/model_checkpoint/model.h5',
    monitor='val_loss',
    save_best_only=True,
    verbose=1
)

earlystop = EarlyStopping(
    monitor='val_loss',
    patience=7,
    verbose=1,
    restore_best_weights=True
)

learning_rate_reduction = ReduceLROnPlateau(
    monitor='val_loss',
    patience=3,
    verbose=1,
    factor=0.5,
    min_lr=0.00001
)

callbacks = [checkpoint, earlystop, learning_rate_reduction]


In [7]:
model.compile(
    optimizer=Adam(),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [8]:
model.fit(
    train_data,
    steps_per_epoch=train_data.samples // train_data.batch_size,
    validation_data=validation_data,
    validation_steps=validation_data.samples // validation_data.batch_size,
    epochs=5,
    callbacks=callbacks
)

Epoch 1/5


1935/1935 [==============================] - ETA: 0s - loss: 0.2124 - accuracy: 0.9149
Epoch 1: val_loss improved from inf to 0.22376, saving model to D:/Yogesh/collage/G12/model_checkpoint\model.h5


c:\Users\yoges\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1935/1935 [==============================] - 512s 263ms/step - loss: 0.2124 - accuracy: 0.9149 - val_loss: 0.2238 - val_accuracy: 0.9006 - lr: 0.0010
Epoch 2/5
1935/1935 [==============================] - ETA: 0s - loss: 0.1835 - accuracy: 0.9273
Epoch 2: val_loss improved from 0.22376 to 0.18594, saving model to D:/Yogesh/collage/G12/model_checkpoint\model.h5
1935/1935 [==============================] - 495s 256ms/step - loss: 0.1835 - accuracy: 0.9273 - val_loss: 0.1859 - val_accuracy: 0.9229 - lr: 0.0010
Epoch 3/5
1935/1935 [==============================] - ETA: 0s - loss: 0.1768 - accuracy: 0.9312
Epoch 3: val_loss did not improve from 0.18594
1935/1935 [==============================] - 522s 270ms/step - loss: 0.1768 - accuracy: 0.9312 - val_loss: 0.1873 - val_accuracy: 0.9255 - lr: 0.0010
Epoch 4/5
1935/1935 [==============================] - ETA: 0s - loss: 0.1721 - accuracy: 0.9332
Epoch 4: val_loss did not improve from 0.18594
1935/1935 [==============================] - 517s

In [9]:
acc_tr,loss_tr =model.evaluate(train_data)
print(acc_tr)
print(loss_tr)

1936/1936 [==============================] - 307s 159ms/step - loss: 0.1522 - accuracy: 0.9407
0.15221057832241058
0.9406541585922241


In [10]:
acc_vr,loss_vr=model.evaluate_generator(validation_data)
print(acc_vr)
print(loss_vr)

C:\Users\yoges\AppData\Local\Temp\ipykernel_11276\4184188003.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  acc_vr,loss_vr=model.evaluate_generator(validation_data)


0.22251105308532715
0.906613290309906


In [11]:
acc_te,loss_te=model.evaluate_generator(test_data)
print(acc_te)
print(loss_te)

C:\Users\yoges\AppData\Local\Temp\ipykernel_11276\3010557615.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  acc_te,loss_te=model.evaluate_generator(test_data)


0.19838464260101318
0.9233136773109436
